In [75]:
import pandas as pd 
import numpy as np
import sklearn as sklearn
import os as os

import matplotlib.pyplot as plt
import seaborn as sns
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import re as re

# if os.getlogin()=="JVARGH7":
#    path_equity_precision_llm_folder = "C:/Cloud/OneDrive - Emory University/Papers/Global Equity in Diabetes Precision Medicine LLM"
#    path_equity_precision_llm_repo =  'C:/code/external/equity_precision_llm'

# elif os.getlogin()=='aamnasoniwala':
#    path_equity_precision_llm_folder = "/Users/aamnasoniwala/Library/CloudStorage/OneDrive-Emory/Global Equity in Diabetes Precision Medicine LLM"
#    path_equity_precision_llm_repo = '/Users/aamnasoniwala/code/equity_precision_llm'

path_equity_precision_llm_folder = "/Users/aamnasoniwala/Library/CloudStorage/OneDrive-Emory/Global Equity in Diabetes Precision Medicine LLM"
path_equity_precision_llm_repo = '/Users/aamnasoniwala/code/equity_precision_llm'

excel_path = path_equity_precision_llm_folder + "/llm training/Test Data Splits/Test Part "
# path_equity_precision_llm_repo = os.path.abspath("").replace("preprocessing", "")

execfile(path_equity_precision_llm_repo + "/functions/crosstab_summary.py")

In [76]:
excel_path_test = path_equity_precision_llm_folder + "/llm training/epldat03_Test Data.xlsx"
# path_equity_precision_llm_repo = os.path.abspath("").replace("preprocessing", "")

execfile(path_equity_precision_llm_repo + "/functions/clean_input.py")
execfile(path_equity_precision_llm_repo + "/functions/crosstab_summary.py")
execfile(path_equity_precision_llm_repo + "/functions/standardize_population.py")
execfile(path_equity_precision_llm_repo + "/functions/adjusted_source_population_match.py")

In [77]:
input_test = pd.read_excel(excel_path_test)
input_test.head()

,PMID,Title,Abstract,MeSH,Source Population,Precision Medicine,Diabetes,Correct Source Population,Primary Study
0,23680249,Pattern and predictors of dyslipidemia in pati...,Dyslipidemia is a major risk factor for macro-...,"Adult, Blood Glucose, Cross-Sectional Studies,...",South Asia,No,Yes,SA,Yes
1,32585310,Proteome-wide assessment of diabetes mellitus ...,Proteomics is expected to provide novel insigh...,"Adult, Blood Proteins, Case-Control Studies, C...",South Asia,No,Yes,SA,Yes
2,34254537,Role of Nutrition Counseling and Lifestyle Mod...,Objective: India is the second country after C...,"Blood Glucose, Counseling, Diabetes Mellitus, ...",South Asia,No,Yes,SA,Yes
3,15287926,Lipids and lipoprotein(a) concentrations in Pa...,The aim of the present study was to analyze se...,"Blood Pressure, Case-Control Studies, Diabetes...",South Asia,No,Yes,SA,Yes
4,16108841,Telomere shortening occurs in Asian Indian Typ...,Telomere shortening has been reported in sever...,"Aging, Cholesterol, Diabetes Mellitus, Type 2,...",South Asia,No,Yes,SA,Yes


In [78]:

n_json_splits = 4
# Concatenate multiple CSVs
results = None
for scenario in range(1, n_json_splits + 1):
    file_path = f"{path_equity_precision_llm_folder}/llm training/Test Data Splits/Test Part {scenario}_results.csv"
    temp_df = pd.read_csv(file_path)
    temp_df = temp_df[['pmid', 'title', 'gpt_precision_medicine', 'gpt_diabetes', 'gpt_primary_study', 'gpt_source_population']]

    if results is None:
        results = temp_df
    else:
        results = pd.concat([results, temp_df], ignore_index=True)

# Merge datasets
merged_df_test = input_test.merge(results, left_on='PMID', right_on='pmid', how='left')
merged_df_test['gpt_source_population'] = merged_df_test['gpt_source_population'].apply(standardize_population)

merged_df_test.to_csv(f"{path_equity_precision_llm_folder}/llm training/Test Scenario 4_results.csv", index=False)


In [79]:
rows, columns = merged_df_test.shape
print(f"Number of rows: {rows}")
print(f"Number of columns: {columns}")

Number of rows: 2091
Number of columns: 15


In [80]:
# File paths
file_names = [
    f"{path_equity_precision_llm_folder}/llm training/Training Scenario 4_results.csv",
    f"{path_equity_precision_llm_folder}/llm training/Development Scenario 4_results.csv",
    f"{path_equity_precision_llm_folder}/llm training/Test Scenario 4_results.csv"
]

# Columns to keep
kept_columns = [
    'pmid', 'title', 'gpt_precision_medicine', 
    'gpt_diabetes', 'gpt_primary_study', 'gpt_source_population'
]

# Read and concatenate all data into a new df
input_all_scenario4 = pd.concat(
    [pd.read_csv(file)[kept_columns] for file in file_names],
    ignore_index=True
)

# Check
print(f"Total rows in new dataframe: {input_all_scenario4.shape[0]}")

# Save df
output_path = f"{path_equity_precision_llm_folder}/llm training/input_all_scenario4.csv"
input_all_scenario4.to_csv(output_path, index=False)

# Apply standardize_population function
input_all_scenario4['gpt_source_population'] = input_all_scenario4['gpt_source_population'].apply(standardize_population)


Total rows in new dataframe: 2211


In [85]:
import pandas as pd

# Define the valid source population categories
VALID_REGIONS = {"ca", "na", "cee", "lac", "sa", "ssa", "we", "seap", "mena", "ea", "Unknown"}
EUROPEAN_REGIONS = {"we", "na"}
NON_EUROPEAN_REGIONS = VALID_REGIONS - EUROPEAN_REGIONS - {"Unknown"}

# Step 1: Identify and count excluded studies due to missing Title or Abstract
def is_missing(value):
    return pd.isna(value) or str(value).strip() == ""

def excluded(row):
    # return is_missing(row['Title'])
    return is_missing(row['title']) 

# Count excluded studies
excluded_studies_df = input_all_scenario4[input_all_scenario4.apply(excluded, axis=1)]
excluded_count = len(excluded_studies_df)
print(f"Number of studies excluded due to missing Abstract: {excluded_count}")

# Apply exclusion
input_all_scenario4_after_exclusion = input_all_scenario4.drop(excluded_studies_df.index)

# Verify remaining studies
remaining_studies = len(input_all_scenario4_after_exclusion)
print(f"Remaining studies after exclusion: {remaining_studies}")

input_all_scenario4_after_exclusion.shape


Number of studies excluded due to missing Abstract: 1
Remaining studies after exclusion: 2210


(2210, 6)

In [86]:
# Step 2: Normalize source population categories

def normalize_source_population(source_population):
    if pd.isna(source_population):
        return set()
    return VALID_REGIONS.intersection(set(re.sub(r'\s+', '', source_population).split(',')))

input_all_scenario4_after_exclusion['normalized_regions'] = input_all_scenario4_after_exclusion['gpt_source_population'].apply(normalize_source_population)

# Count rows where normalized_regions has more than 1 element
multi_region_rows = input_all_scenario4_after_exclusion[input_all_scenario4_after_exclusion['normalized_regions'].apply(lambda x: len(x) > 1)]
multi_region_count = len(multi_region_rows)
print(f"Number of rows where normalized_regions has more than 1 element: {multi_region_count}")

Number of rows where normalized_regions has more than 1 element: 123


In [87]:
# Step 3: Expand data **only** for regional counts
expanded_rows = []
for _, row in input_all_scenario4_after_exclusion.iterrows():
    for region in row['normalized_regions']:
        new_row = row.copy()
        new_row['gpt_source_population'] = region
        expanded_rows.append(new_row)

# Create expanded DataFrame for regional counts
expanded_df = pd.DataFrame(expanded_rows)
expanded_df.shape

(2447, 7)

In [88]:
# Step 4: Create a crosstab summary function
def crosstab_summary(expanded_df, categories, region_col='gpt_source_population'):
    expanded_df['combined_category'] = expanded_df[categories].astype(str).agg('_'.join, axis=1)
    summary = pd.crosstab(expanded_df['combined_category'], expanded_df[region_col])
    summary.reset_index(inplace=True)
    return summary

# Define categories
categories = ['gpt_precision_medicine', 'gpt_diabetes', 'gpt_primary_study']

# Generate crosstab for regional counts
summary_expanded_df = crosstab_summary(expanded_df, categories, region_col='gpt_source_population')

# Step 5: Compute overall totals without double-counting studies
total_counts = {
    'Total Precision Medicine': input_all_scenario4_after_exclusion['gpt_precision_medicine'].notna().sum(),
    'Total Diabetes': input_all_scenario4_after_exclusion['gpt_diabetes'].notna().sum(),
    'Total Primary Study': input_all_scenario4_after_exclusion['gpt_primary_study'].notna().sum()
}

total_counts

{'Total Precision Medicine': np.int64(2210),
 'Total Diabetes': np.int64(2210),
 'Total Primary Study': np.int64(2210)}

In [90]:
# Step 6: Save the expanded output for regions
output_path = path_equity_precision_llm_repo + '/preprocessing/2202 manually screened.csv'
summary_expanded_df.to_csv(output_path, index=False)

# Display summary
display(summary_expanded_df)

gpt_source_population,combined_category,Unknown,ca,cee,ea,lac,mena,na,sa,seap,ssa,we
0,no_no_no,81,1,1,9,7,2,19,5,1,4,10
1,no_no_yes,215,6,51,70,67,18,50,26,20,9,25
2,no_yes_no,1,2,5,13,1,1,11,15,3,1,9
3,no_yes_not determined,1,0,0,0,0,0,0,0,0,0,0
4,no_yes_yes,48,3,30,88,52,36,56,50,35,16,30
5,yes_no_no,3,0,0,1,3,0,5,4,0,0,4
6,yes_no_unknown,0,0,0,0,1,0,0,0,0,0,0
7,yes_no_yes,16,0,13,69,67,7,34,33,13,4,12
8,yes_yes_no,0,0,1,9,10,0,12,13,3,2,4
9,yes_yes_not determinable,0,0,0,0,0,0,0,1,0,0,0


In [91]:
# Step 7: Categorize studies into European vs. Non-European vs. Unknown (without duplicates)
def classify_region(regions):
    classification = set()
    if any(r in EUROPEAN_REGIONS for r in regions):
        classification.add('European')
    if any(r in NON_EUROPEAN_REGIONS for r in regions):
        classification.add('Non-European')
    if not classification:
        classification.add('Unknown')
    return classification

# Remove rows where gpt_source_population is 'Unknown'


input_all_scenario4_after_exclusion['classification_group'] = input_all_scenario4_after_exclusion['normalized_regions'].apply(classify_region)

input_all_scenario4_after_exclusion_region = input_all_scenario4_after_exclusion[input_all_scenario4_after_exclusion['gpt_source_population'] != 'Unknown']

# Compute total counts per classification group
group_totals = input_all_scenario4_after_exclusion_region['classification_group'].value_counts()

In [92]:

expanded_rows_region = []
for _, row in input_all_scenario4_after_exclusion_region.iterrows():
    for group in row['classification_group']:
        new_row = row.copy()
        new_row['classification_group'] = group
        expanded_rows_region.append(new_row)

# Create expanded DataFrame for regional counts
expanded_df_region = pd.DataFrame(expanded_rows_region)
expanded_df_region.shape


summary_expanded_df_region = crosstab_summary(expanded_df_region, categories, region_col='classification_group')

In [93]:
# Step 8: Save the expanded output for regions
output_path_group = path_equity_precision_llm_repo + '/preprocessing/2202 manually screened.csv'
summary_expanded_df_region.to_csv(output_path_group, index=False)

# Display summary
display(summary_expanded_df_region)

classification_group,combined_category,European,Non-European
0,no_no_no,24,25
1,no_no_yes,74,257
2,no_yes_no,18,33
3,no_yes_yes,83,297
4,yes_no_no,7,8
5,yes_no_unknown,0,1
6,yes_no_yes,42,194
7,yes_yes_no,12,27
8,yes_yes_not determinable,0,1
9,yes_yes_yes,176,637
